In [1]:
from datetime import datetime
from  os import name, path
from io import StringIO
from time import sleep
import json

import utils
from pipe import sort
import pprint 
from AvailableBalances import AvailableBalance
from Ledgers import Ledgers
import Orders
import Trades  
import pandas as pd

In [2]:
av = AvailableBalance()
bal = av.get_avalailable_balances().sort_values('Currency')
balanceResult = bal[bal['Balance']>0.1]
print(balanceResult)


   Currency            Balance
16      ADA      1199.49426473
20    MATIC       2.8903747200
3       TRX         8.89447921
29     XXLM         8.23315119
22     XXRP         0.78177634
26     ZEUR  1670.732999996286


### Get balances together with currency details

In [3]:
marketData = pd.DataFrame(av.Market)
merged = pd.merge(marketData, balanceResult,left_on='name',right_on='Currency')
merged.drop(columns=["Currency","aclass"])

,altname,decimals,display_decimals,name,Balance
0,ADA,8,6,ADA,1199.49426473
1,MATIC,10,5,MATIC,2.8903747200
2,TRX,8,6,TRX,8.89447921
3,XLM,8,5,XXLM,8.23315119
4,XRP,8,5,XXRP,0.78177634
5,EUR,4,2,ZEUR,1670.732999996286


### Download history trade data
- report_id will be persisted in ./data/report_<ledger|trades>.json
- if report_id already available, download report from API
- if there is no report_id, request report at kraken and wait for being ready for download

In [4]:
print()
print(f'trades history:')
print("-------------------")
th = Trades.TradeHistory()
dataTrades = th.get_from_Export(startDate=utils.date2nix(datetime(2021,1,1)))
print(dataTrades.head())


trades history:
-------------------
{'ofs': 0, 'trades': True}
Open Zip-File fore reading trades
                  txid            ordertxid    pair                      time  \
0  TYG3ZQ-7TPTH-C53PEY  OEWOU3-YY3OC-YK7EVN  DOTEUR   2021-02-13 08:05:20.067   
1  TZCJGJ-VA5S4-7RLUKZ  OQGKP5-WNE4Q-ABADSG  DOTEUR  2021-02-23 09:08:11.8285   
2  TLBXYT-GNG2O-W3RJDC  OUDGXV-SG7MW-P4GYJ7  DOTEUR  2021-02-23 11:17:21.1323   
3  TBUJAA-75K4V-RPLWA3  OIBAPB-SUNTR-XPBDAE  DOTEUR  2021-02-23 13:46:23.5847   
4  TFGX2M-M7B6A-SMD334  OUIQQD-EQN34-EVW6UA  DOTEUR  2021-02-23 14:19:05.1431   

   type ordertype     price       cost       fee         vol  ...  \
0   buy     limit  24.00000  3993.6102   6.38978  166.400425  ...   
1   buy     limit  23.50000  3525.0000   6.34500  150.000000  ...   
2  sell    market  25.67218  3850.8270  10.01215  150.000000  ...   
3  sell     limit  27.00000    27.0000   0.04860    1.000000  ...   
4   buy     limit  28.20000    28.2000   0.04512    1.000000  ...   



### Download history ledger data

In [5]:
print()
print(f'ledgers history:')
print("-------------------")
ledger = Ledgers()
dataLedger = ledger.get_from_Export(startDate=utils.date2nix(datetime(2021,1,1)))
print(dataLedger.head())


ledgers history:
-------------------
{'ofs': 0}
Open Zip-File fore reading ledgers
                  txid                  refid                 time     type  \
0                  NaN  QCCD2WM-SDW3XQ-UL7BIY  2021-02-11 06:12:28  deposit   
1  LA5JWG-DL5YT-NJ3CDF  QCCD2WM-SDW3XQ-UL7BIY  2021-02-11 06:13:27  deposit   
2  LAAS5I-DGCZF-U62UH2    TYG3ZQ-7TPTH-C53PEY  2021-02-13 08:05:20    trade   
3  L35LIS-UQ7JF-TJPH7G    TYG3ZQ-7TPTH-C53PEY  2021-02-13 08:05:20    trade   
4                  NaN  QCCZW7J-JVI25W-OOXEM3  2021-02-23 08:07:22  deposit   

   subtype    aclass asset       amount     fee      balance  
0      NaN  currency  ZEUR  4000.000000  0.0000          NaN  
1      NaN  currency  ZEUR  4000.000000  0.0000  4000.000000  
2      NaN  currency  ZEUR -3993.610200  6.3898     0.000000  
3      NaN  currency   DOT   166.400425  0.0000   166.400425  
4      NaN  currency  ZEUR  4000.000000  0.0000          NaN  


### Filter ledger data for currency from balance data

In [34]:
if(not dataLedger is None):
    mergedForBalance=pd.merge(dataLedger, balanceResult, left_on="asset",right_on="Currency", how="left").sort_values(by="time")
    mergedForBalance=mergedForBalance[~mergedForBalance["type"].isin( ["rollover","margin"])]
    mergedWithTrades = pd.merge(mergedForBalance,dataTrades["txid","type"],left_on="refid",right_on="txid")
    mergedWithTrades.reindex(mergedForBalance.index)

KeyError: ('txid', 'type')